# Analyse IPL 2022 Auction Data 

Using Pandas and Pyspark SQL

In [80]:
# # import pandassql
# !pip install pandasql

In [81]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *

Create Spark session

In [82]:
spark = SparkSession.builder.master('local[*]').appName("IPL2022Auction").config("spark.driver.bindAddress","10.0.2.15").getOrCreate()


Read Dataset

In [83]:
df = spark.read.csv("ipl_2022_dataset.csv",header=True,inferSchema=True)

df.show()

23/08/11 15:28:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Player, Base Price, TYPE, COST IN ₹ (CR.), Cost IN $ (000), 2021 Squad, Team
 Schema: _c0, Player, Base Price, TYPE, COST IN ₹ (CR.), Cost IN $ (000), 2021 Squad, Team
Expected: _c0 but found: 
CSV file: file:///home/hdoop/Documents/python/Pyspark_Analysis/ipl_2022_dataset.csv
+---+-------------------+----------+------------+---------------+---------------+----------+--------------+
|_c0|             Player|Base Price|        TYPE|COST IN ₹ (CR.)|Cost IN $ (000)|2021 Squad|          Team|
+---+-------------------+----------+------------+---------------+---------------+----------+--------------+
|  0|        Rashid Khan|Draft Pick|      BOWLER|           15.0|         1950.0|       SRH|Gujarat Titans|
|  1|      Hardik Pandya|Draft Pick| ALL-ROUNDER|           15.0|         1950.0|        MI|Gujarat Titans|
|  2|    Lockie Ferguson|      2 Cr|      BOWLER|           10.0|         1300.0|      

check the data type of columns

In [84]:
type('_c0')

str

# Data Exploration

In [85]:
print("Total Number of Rows:",df.count())
print("Total number of columns:",len(df.columns))

Total Number of Rows: 633
Total number of columns: 8


Describe Dataframe

In [86]:
df.describe().summary().show()

23/08/11 15:29:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Player, Base Price, TYPE, COST IN ₹ (CR.), Cost IN $ (000), 2021 Squad, Team
 Schema: _c0, Player, Base Price, TYPE, COST IN ₹ (CR.), Cost IN $ (000), 2021 Squad, Team
Expected: _c0 but found: 
CSV file: file:///home/hdoop/Documents/python/Pyspark_Analysis/ipl_2022_dataset.csv
+-------+-------+------------------+------------+----------+------------+------------------+------------------+----------+------+
|summary|summary|               _c0|      Player|Base Price|        TYPE|   COST IN ₹ (CR.)|   Cost IN $ (000)|2021 Squad|  Team|
+-------+-------+------------------+------------+----------+------------+------------------+------------------+----------+------+
|  count|      5|                 5|           3|         3|           3|                 5|                 5|         3|     3|
|   mean|   null| 352.7751281610878|       633.0|     633.0|       633.0| 52.42677449451607| 700.8806842870

Data types of each column

In [87]:
# Data types
df.dtypes

[('_c0', 'int'),
 ('Player', 'string'),
 ('Base Price', 'string'),
 ('TYPE', 'string'),
 ('COST IN ₹ (CR.)', 'double'),
 ('Cost IN $ (000)', 'double'),
 ('2021 Squad', 'string'),
 ('Team', 'string')]

In [88]:
# unique values in 'Base Price' colum
unique_values_base_price = df.select('Base Price').distinct()
unique_values_base_price.show()

+----------+
|Base Price|
+----------+
|   20 Lakh|
|      2 Cr|
|Draft Pick|
|   40 Lakh|
|   30 Lakh|
|      1 Cr|
|   50 Lakh|
|    1.5 Cr|
|   75 Lakh|
|  Retained|
+----------+



In [89]:
# Distribution of values inside column 'Base Price'
base_price_value_count = df.groupBy('Base Price').count()
base_price_value_count.orderBy(desc('count')).show()

+----------+-----+
|Base Price|count|
+----------+-----+
|   20 Lakh|  344|
|   50 Lakh|  104|
|      2 Cr|   48|
|      1 Cr|   33|
|  Retained|   27|
|   75 Lakh|   26|
|    1.5 Cr|   20|
|   40 Lakh|   16|
|   30 Lakh|    9|
|Draft Pick|    6|
+----------+-----+



In [90]:
# unique values in 'Type' column
type_distribution = df.groupBy("TYPE").count()
type_distribution.orderBy(desc("count")).show()

+------------+-----+
|        TYPE|count|
+------------+-----+
| ALL-ROUNDER|  242|
|      BOWLER|  215|
|      BATTER|  112|
|WICKETKEEPER|   64|
+------------+-----+



Question to Answer
1. Top 3 batsmen who get paid the most?
2. Top 5 bowlers who get maid the most?
3. Highest paid All-rounders?
4. Average pay for batsman,Wicket Keeper?
5. List of Retained Players with salary?

## Data Transformation

Dataframe manipulation 
1. Rename columns

In [91]:
df.columns

['_c0',
 'Player',
 'Base Price',
 'TYPE',
 'COST IN ₹ (CR.)',
 'Cost IN $ (000)',
 '2021 Squad',
 'Team']

Rename column names in dataframe

In [92]:
# rename columns and save it in variable df2
df_rename_columns = {'Player':'player',
                   'Base Price':'base_price',
                   'TYPE':'type',
                   'COST IN ₹ (CR.)':'cost_inr',
                   'Cost IN $ (000)':'cost_usd',
                    '2021 Squad':'old_team',
                    'Team':'new_team'
                   }

for old_column_name ,new_column_name in df_rename_columns.items():
    df = df.withColumnRenamed(old_column_name,new_column_name)

In [93]:
df.show()

23/08/11 15:29:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Player, Base Price, TYPE, COST IN ₹ (CR.), Cost IN $ (000), 2021 Squad, Team
 Schema: _c0, Player, Base Price, TYPE, COST IN ₹ (CR.), Cost IN $ (000), 2021 Squad, Team
Expected: _c0 but found: 
CSV file: file:///home/hdoop/Documents/python/Pyspark_Analysis/ipl_2022_dataset.csv
+---+-------------------+----------+------------+--------+--------+--------+--------------+
|_c0|             player|base_price|        type|cost_inr|cost_usd|old_team|      new_team|
+---+-------------------+----------+------------+--------+--------+--------+--------------+
|  0|        Rashid Khan|Draft Pick|      BOWLER|    15.0|  1950.0|     SRH|Gujarat Titans|
|  1|      Hardik Pandya|Draft Pick| ALL-ROUNDER|    15.0|  1950.0|      MI|Gujarat Titans|
|  2|    Lockie Ferguson|      2 Cr|      BOWLER|    10.0|  1300.0|     KKR|Gujarat Titans|
|  3|      Rahul Tewatia|   40 Lakh| ALL-ROUNDER|     9.0|  1170.0|      RR

In [94]:
# Dropping USD column
df2 = df.drop('cost_usd')
df2.show()

23/08/11 15:29:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Player, Base Price, TYPE, COST IN ₹ (CR.), 2021 Squad, Team
 Schema: _c0, Player, Base Price, TYPE, COST IN ₹ (CR.), 2021 Squad, Team
Expected: _c0 but found: 
CSV file: file:///home/hdoop/Documents/python/Pyspark_Analysis/ipl_2022_dataset.csv
+---+-------------------+----------+------------+--------+--------+--------------+
|_c0|             player|base_price|        type|cost_inr|old_team|      new_team|
+---+-------------------+----------+------------+--------+--------+--------------+
|  0|        Rashid Khan|Draft Pick|      BOWLER|    15.0|     SRH|Gujarat Titans|
|  1|      Hardik Pandya|Draft Pick| ALL-ROUNDER|    15.0|      MI|Gujarat Titans|
|  2|    Lockie Ferguson|      2 Cr|      BOWLER|    10.0|     KKR|Gujarat Titans|
|  3|      Rahul Tewatia|   40 Lakh| ALL-ROUNDER|     9.0|      RR|Gujarat Titans|
|  4|       Shubman Gill|Draft Pick|      BATTER|     8.0|     KKR|Gujarat Titan

# setting pyspark sql

In [95]:
df2.createOrReplaceTempView("sqldf")

In [96]:
spark.sql("select * from sqldf").show()

23/08/11 15:29:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Player, Base Price, TYPE, COST IN ₹ (CR.), 2021 Squad, Team
 Schema: _c0, Player, Base Price, TYPE, COST IN ₹ (CR.), 2021 Squad, Team
Expected: _c0 but found: 
CSV file: file:///home/hdoop/Documents/python/Pyspark_Analysis/ipl_2022_dataset.csv
+---+-------------------+----------+------------+--------+--------+--------------+
|_c0|             player|base_price|        type|cost_inr|old_team|      new_team|
+---+-------------------+----------+------------+--------+--------+--------------+
|  0|        Rashid Khan|Draft Pick|      BOWLER|    15.0|     SRH|Gujarat Titans|
|  1|      Hardik Pandya|Draft Pick| ALL-ROUNDER|    15.0|      MI|Gujarat Titans|
|  2|    Lockie Ferguson|      2 Cr|      BOWLER|    10.0|     KKR|Gujarat Titans|
|  3|      Rahul Tewatia|   40 Lakh| ALL-ROUNDER|     9.0|      RR|Gujarat Titans|
|  4|       Shubman Gill|Draft Pick|      BATTER|     8.0|     KKR|Gujarat Titan

1. Top 3 batsmen who get paid the most?

In [97]:
spark.sql("select player,cost_inr from sqldf where type='BATTER' ORDER BY cost_inr DESC LIMIT 3").show()

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|   Rohit Sharma|    16.0|
|    Virat Kohli|    15.0|
|Kane Williamson|    14.0|
+---------------+--------+



2. Top 5 bowlers who get paid the most?

In [98]:
spark.sql("select player,cost_inr from sqldf where type='BOWLER' ORDER BY cost_inr DESC LIMIT 5 ").show()

+--------------+--------+
|        player|cost_inr|
+--------------+--------+
|   Rashid Khan|    15.0|
| Deepak Chahar|    14.0|
|Jasprit Bumrah|    12.0|
|Shardul Thakur|   10.75|
|    Avesh Khan|    10.0|
+--------------+--------+



3. Highest paid All-rounders

In [99]:
spark.sql("select player,cost_inr from sqldf where type='ALL-ROUNDER' ORDER BY cost_inr DESC LIMIT 1 ").show()

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|Ravindra Jadeja|    16.0|
+---------------+--------+



4. Top 5 lowest paid wicket keeper

In [100]:
spark.sql("""select player,cost_inr from sqldf where type='WICKETKEEPER' AND cost_inr is not null ORDER BY cost_inr LIMIT 5""").show()

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|    Dhruv Jurel|     0.2|
|  Jitesh Sharma|     0.2|
|  N. Jagadeesan|     0.2|
|    Aryan Juyal|     0.2|
|Luvnith Sisodia|     0.2|
+---------------+--------+



4. Average pay for batsman,Wicket Keeper?

In [101]:
spark.sql("select type,round(avg(cost_inr),2) as Average_cost_inr from sqldf GROUP BY TYPE ORDER BY 2 DESC").show()

+------------+----------------+
|        type|Average_cost_inr|
+------------+----------------+
|WICKETKEEPER|            5.09|
|      BATTER|            4.11|
| ALL-ROUNDER|            3.61|
|      BOWLER|            3.07|
+------------+----------------+



5. List of retained players with team name and salary

In [105]:
spark.sql("select player,cost_inr,new_team as Retained_team from sqldf where base_price='Retained' ORDER BY cost_inr DESC LIMIT 5").show()

+---------------+--------+--------------------+
|         player|cost_inr|       Retained_team|
+---------------+--------+--------------------+
|Ravindra Jadeja|    16.0| Chennai Super Kings|
|   Rishabh Pant|    16.0|      Delhi Capitals|
|   Rohit Sharma|    16.0|      Mumbai Indians|
|    Virat Kohli|    15.0|Royal Challengers...|
|   Sanju Samson|    14.0|    Rajasthan Royals|
+---------------+--------+--------------------+



close Spark session

In [108]:
spark.stop()